# Costruzioni dataset

In [23]:
import os
import sentencepiece as spm
import pandas as pd
import numpy as np
import xml.etree.ElementTree as ET
import xml.dom.minidom as minidom

## Variabili

In [24]:
lingua_dataset = 'it-PE'
tipo_dataset_generato = 'TCO'
random_number = str(np.random.randint(0, 100000))
flag_shape = True
flag_shape_type = True
flag_size = True

## Dataset per RDF2Text

### Train e dev set

In [25]:
dataset_types = ["train", "dev"] 
triple_numbers = ["1", "2", "3", "4", "5", "6", "7"]

i = 0
for dataset_type in dataset_types:
    dataset = []

    for triple_number in triple_numbers:
        path = "..\\..\\webnlg\\it-PE\\" + dataset_type + "\\" + triple_number + "triples"
        file_names = []
        for file_name in os.listdir(path):
            if os.path.isfile(os.path.join(path, file_name)):
                i += 1
                url = os.path.join(path, file_name)
                tree = ET.parse(url)
                root = tree.getroot()

                for entry in root.iter('entry'):
                    mts = []
                    eid = entry.get('eid')
                    category = entry.get('category')
                    size = entry.get('size')

                    if entry.get('shape_type') is not None:
                        shape_type = entry.get('shape_type')
                    else:
                        shape_type = "NA"
                        print("Shape type not found in entry " + eid)

                    mts = []                    
                    for modifiedtripleset in entry.iter('modifiedtripleset'):
                        mtriples = ""

                        triples = []
                        for mtriple in modifiedtripleset.iter('mtriple'):  
                            triple = mtriple.text.split(" | ")
                            triple = (triple[0], triple[1], triple[2])
                            triples.append(triple)

                        triples.sort(key=lambda x: x[1])
                        for triple in triples:
                            mtriples += triple[0] + " " + triple[1] + " " + triple[2] + " "

                        # if last character is space remove it
                        if mtriples[-1] == " ":
                            mtriples = mtriples[:-1]

                        mts.append(mtriples)

                    lexs = []
                    for lex in entry.iter('lex'):
                        if lex.get('lang') == 'it-PE':
                            lexs.append(lex.text)

                    for mt in mts:
                        for lex in lexs:
                            i += 1
                            dataset.append([eid, size, category, shape_type, mt, lex])

    dataset_pd = pd.DataFrame(dataset, columns = ['eid', 'size', 'category', 'shape_type', 'data_unit', 'sentence'])
    dataset_pd.to_csv(f"{dataset_type}_0.csv", index=False)

### Test set

In [26]:
dataset = []

i = 0

path = "..\\..\\WebNLG\\it-PE\\test"
file_names = []
for file_name in os.listdir(path):
    if (file_name == 'rdf-to-text-generation-test-data-with-refs-en.xml'):
        if os.path.isfile(os.path.join(path, file_name)):
            i += 1
            url = os.path.join(path, file_name)
            tree = ET.parse(url)
            root = tree.getroot()

            for entry in root.iter('entry'):
                mts = []
                eid = entry.get('eid')
                category = entry.get('category')
                size = entry.get('size')

                if entry.get('shape_type') is not None:
                    shape_type = entry.get('shape_type')
                else:
                    shape_type = "NA"

                for modifiedtripleset in entry.iter('modifiedtripleset'):
                    mtriples = ""

                    triples = []
                    for mtriple in modifiedtripleset.iter('mtriple'):  
                        triple = mtriple.text.split(" | ")
                        triple = (triple[0], triple[1], triple[2])
                        triples.append(triple)

                    triples.sort(key=lambda x: x[1])
                    for triple in triples:
                        mtriples += triple[0] + " " + triple[1] + " " + triple[2] + " | "

                   # Rimuovi l'ultimo " | " se presente
                    if mtriples.endswith(" | "):
                        mtriples = mtriples[:-3]

                    mts.append(mtriples)

                lexs = []
                for lex in entry.iter('lex'):
                    if lex.get('lang') == 'it-PE':
                        lexs.append(lex.text)

                for mt in mts:
                    i += 1
                    dataset.append([eid, size, category, shape_type, mt, lexs])

dataset_pd = pd.DataFrame(dataset, columns = ['eid', 'size', 'category', 'shape_type', 'data_unit', 'sentence'])
dataset_pd = dataset_pd.sample(frac=1, random_state=42).reset_index(drop=True)
dataset_pd.to_csv(f"test_0.csv", index=False)